## Загрузим нужные библиотеки

In [41]:
#!pip install img2vec_pytorch

In [228]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from os import listdir 
from img2vec_pytorch import Img2Vec
from PIL import Image
from pathlib import Path
import glob
import pickle
#from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial


import torch
from transformers import AutoTokenizer, AutoModel

%matplotlib inline

In [71]:
SEED = 1

## Создаем эмбеддинги для тренировочного и для тестового набора картинок

In [72]:
home = "./content"
dataset = "train" # "test"
path_images = f"{home}/{dataset}"
#img2vecEffb7 = Img2Vec(cuda=False, model="efficientnet_b7")
img2vec = Img2Vec(cuda=False)
#path_tfrecords = f"{home}/{dataset}/tfrecords"
#path_embeddings = f"{home}/{dataset}/embeddings"
path_images

'./content/train'

In [77]:
from tqdm import tqdm

def get_image_embeddings(path_images):
    img_emb = dict()
    emb_img = dict()
    for file in tqdm(glob.glob(f"{path_images}/*.png")):
        img = Image.open(file).convert('RGB')
        #vec = img2vecEffb7.get_vec(img, tensor=True)
        vec = img2vec.get_vec(img, tensor=False)
        #print(vec)
        filename = file.split("\\")[1].split(".")[0]
        img_emb[filename] = vec
        #emb_img[vec] = filename
    return img_emb#, emb_img

In [78]:
dataset = "train" 
path_images = f"{home}/{dataset}"
#train_img_emb, train_emb_img = get_image_embeddings(path_images)
train_img_emb = get_image_embeddings(path_images)

100%|██████████████████████████████████████████████████████████████████████████████| 2098/2098 [02:33<00:00, 13.67it/s]


In [79]:
testdataset = "test" 
test_path_images = f"{home}/{testdataset}"
test_img_emb = get_image_embeddings(test_path_images)

100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:55<00:00, 16.30it/s]


In [80]:
with open(f'{home}/saved_train_img_emb.pkl', 'wb') as f:
    pickle.dump(train_img_emb, f)
#with open(f'{home}/saved_train_emb_img.pkl', 'wb') as f:
#    pickle.dump(train_emb_img, f)    
with open(f'{home}/saved_test_img_emb.pkl', 'wb') as f:
    pickle.dump(test_img_emb, f)
#with open(f'{home}/saved_test_emb_img.pkl', 'wb') as f:
#    pickle.dump(test_emb_img, f)        

In [81]:
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

## Подготавливаем русскоязычный энкодер предложений (https://habr.com/ru/post/669674/)

In [82]:
# pip install transformers sentencepiece
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [83]:
print(embed_bert_cls('Фотонегатив пленочный. Серебряная чаша из Хорезма (III-IV вв.). Найдена в деревне Бартым, Березовского района, Пермской области.  ПОКМ-18204/16 фотосъемка,фотопленка    Тартаковский Сергей А. (Фотограф) Изображение чаши с чеканным рельефом на зеркале (фигура верхом на льве). Вид сверху.', model, tokenizer))

[ 7.29140416e-02 -2.70076636e-02 -8.55205301e-03 -3.95895317e-02
 -2.90382337e-02 -1.72058977e-02 -2.95073483e-02  5.50180823e-02
 -2.96006799e-02  7.65557066e-02  1.67463254e-03 -9.10883304e-03
  2.11446229e-02  7.27286562e-02  2.61101369e-02 -4.64161038e-02
  9.97841638e-03 -2.19794977e-02  3.20401154e-02 -1.07474178e-02
 -2.76477393e-02  1.97625672e-03  1.83698692e-04 -4.26389500e-02
  7.93270692e-02  1.73863210e-03  3.88623029e-02 -2.18212586e-02
  9.43261478e-03  5.37685491e-03  3.55617180e-02  1.85357034e-02
  7.64380470e-02  3.88181880e-02 -7.89109617e-02  1.25940815e-02
 -3.90350306e-03 -3.07981018e-02  3.98940183e-02  7.91633222e-03
 -7.67934509e-03  1.84255749e-01  2.52116174e-02  2.63527092e-02
 -4.50597629e-02 -8.31032842e-02 -3.13605964e-02 -4.24337983e-02
  2.70749126e-02 -4.00372110e-02 -1.09781250e-02 -3.25138606e-02
  1.77764483e-02  4.70524468e-02 -8.94650966e-02  5.04260734e-02
  2.51897192e-03  3.85735929e-03  2.98845917e-02  1.22798122e-02
 -3.72947864e-02  5.48969

## Загружаем датасет и готовим текстовый эмбеддинг для обучения

In [84]:
df = pd.read_csv("./content/train.csv")

In [85]:
def embed_bert_cls0(text):
    return embed_bert_cls(text, model, tokenizer)

In [86]:
df['txt_emb'] = df['description'].apply(embed_bert_cls0)

In [87]:
df.head()

,id,description,object_img,txt_emb
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,"[0.036625743, -0.03603049, -0.028768867, -0.00..."
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,"[0.012770161, -0.008171949, -0.051046774, -0.0..."
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,"[0.027026305, 0.010177814, 0.009805401, -0.044..."
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526,"[0.07291404, -0.027007664, -0.008552053, -0.03..."
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244,"[0.03678191, -0.035710353, -0.017911453, -0.02..."


In [90]:
def embed_image(object_img):
    return train_img_emb[str(object_img)]

In [91]:
df['img_emb'] = df['object_img'].apply(embed_image)

In [92]:
df.head()

,id,description,object_img,txt_emb,img_emb
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,"[0.036625743, -0.03603049, -0.028768867, -0.00...","[0.13447547, 2.1468716, 3.0936837, 0.25017983,..."
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,"[0.012770161, -0.008171949, -0.051046774, -0.0...","[0.41817018, 2.1244812, 0.7217666, 1.2344979, ..."
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,"[0.027026305, 0.010177814, 0.009805401, -0.044...","[0.80793715, 0.23533988, 0.8915884, 2.1188881,..."
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526,"[0.07291404, -0.027007664, -0.008552053, -0.03...","[1.123798, 0.12910706, 0.52993155, 0.68911886,..."
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244,"[0.03678191, -0.035710353, -0.017911453, -0.02...","[0.24921188, 1.3386794, 1.4989858, 0.32577786,..."


In [93]:
df = df.set_index('id')

## Обучаем модель

In [159]:
def unpack_column(df, col, num_new_cols):
    
    col_list = [f'{col}_{i:03}' for i in range(num_new_cols)]
    
    val_col = df[col]
    
    df = pd.concat([df,pd.DataFrame(columns=col_list)])
    
    for j in tqdm(range(len(val_col))):
        for i in range(1, num_new_cols+1):
            df.iloc[j, i] = val_col.iloc[j][i-1]
    return df

In [160]:
X = df[['txt_emb']]
X = unpack_column(X, 'txt_emb', 312)
X = X.drop('txt_emb', axis=1)
y = df[['img_emb']]
y = unpack_column(y, 'img_emb', 512)
y = y.drop('img_emb', axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 2098/2098 [01:05<00:00, 32.08it/s]


In [161]:
X

,txt_emb,txt_emb_000,txt_emb_001,txt_emb_002,txt_emb_003,txt_emb_004,txt_emb_005,txt_emb_006,txt_emb_007,txt_emb_008,...,txt_emb_302,txt_emb_303,txt_emb_304,txt_emb_305,txt_emb_306,txt_emb_307,txt_emb_308,txt_emb_309,txt_emb_310,txt_emb_311
520,"[0.036625743, -0.03603049, -0.028768867, -0.00...",0.036626,-0.03603,-0.028769,-0.00336,-0.015004,-0.033309,-0.054957,-0.067121,-0.003853,...,-0.012743,0.060545,-0.017607,-0.052125,-0.104249,0.029912,0.058745,0.047537,0.036392,-0.085036
1817,"[0.012770161, -0.008171949, -0.051046774, -0.0...",0.01277,-0.008172,-0.051047,-0.052968,-0.034557,-0.099109,-0.0217,-0.070497,0.015416,...,-0.017023,0.059315,0.042362,-0.034292,-0.12735,0.059061,-0.036284,0.059862,0.05609,-0.10466
188,"[0.027026305, 0.010177814, 0.009805401, -0.044...",0.027026,0.010178,0.009805,-0.044877,0.039475,-0.017286,-0.012254,0.044032,-0.010038,...,-0.002072,0.012146,0.016421,0.040288,-0.071445,0.020756,-0.024079,0.050283,0.045011,-0.085473
1472,"[0.07291404, -0.027007664, -0.008552053, -0.03...",0.072914,-0.027008,-0.008552,-0.03959,-0.029038,-0.017206,-0.029507,0.055018,-0.029601,...,-0.018427,0.051107,-0.040179,0.043072,-0.14353,0.057518,0.007767,0.100975,0.032745,-0.073464
1664,"[0.03678191, -0.035710353, -0.017911453, -0.02...",0.036782,-0.03571,-0.017911,-0.028027,0.004601,0.021264,-0.017391,0.014238,0.024106,...,0.04745,0.041513,0.044099,0.03981,-0.056193,0.075596,-0.00476,0.034916,-0.040752,-0.114606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,"[0.012695988, -0.039128415, -0.060502604, -0.0...",0.012696,-0.039128,-0.060503,-0.044415,-0.029829,-0.073618,-0.050863,-0.032358,0.009092,...,0.008174,0.035563,0.044137,-0.014154,-0.14099,0.049724,-0.020325,0.029315,0.048436,-0.096676
1297,"[0.067530654, 0.02787305, -0.034303114, -0.036...",0.067531,0.027873,-0.034303,-0.036806,0.042337,-0.014571,0.047287,0.043312,-0.055092,...,-0.019503,0.000634,-0.02769,-0.018206,-0.05588,0.05599,0.013589,0.055061,0.057517,-0.15575
1507,"[0.11323303, -0.016260047, -0.039521314, 0.003...",0.113233,-0.01626,-0.039521,0.003618,0.009733,-0.006297,-0.057463,-0.06061,0.013667,...,0.038726,0.058563,-0.015468,-0.078145,-0.074099,0.006992,0.033966,0.022923,0.035547,-0.073296
1863,"[0.015523672, 0.0032938614, 0.04388629, -0.057...",0.015524,0.003294,0.043886,-0.057055,0.00905,0.015464,0.005539,0.021111,-0.016344,...,0.042147,0.004722,-0.007199,0.010069,0.006064,0.021039,-0.036758,0.069408,-0.010019,-0.128117


In [164]:
X = X.drop('txt_emb', axis=1)

In [165]:
X

,txt_emb_000,txt_emb_001,txt_emb_002,txt_emb_003,txt_emb_004,txt_emb_005,txt_emb_006,txt_emb_007,txt_emb_008,txt_emb_009,...,txt_emb_302,txt_emb_303,txt_emb_304,txt_emb_305,txt_emb_306,txt_emb_307,txt_emb_308,txt_emb_309,txt_emb_310,txt_emb_311
520,0.036626,-0.03603,-0.028769,-0.00336,-0.015004,-0.033309,-0.054957,-0.067121,-0.003853,0.019704,...,-0.012743,0.060545,-0.017607,-0.052125,-0.104249,0.029912,0.058745,0.047537,0.036392,-0.085036
1817,0.01277,-0.008172,-0.051047,-0.052968,-0.034557,-0.099109,-0.0217,-0.070497,0.015416,-0.024679,...,-0.017023,0.059315,0.042362,-0.034292,-0.12735,0.059061,-0.036284,0.059862,0.05609,-0.10466
188,0.027026,0.010178,0.009805,-0.044877,0.039475,-0.017286,-0.012254,0.044032,-0.010038,0.032753,...,-0.002072,0.012146,0.016421,0.040288,-0.071445,0.020756,-0.024079,0.050283,0.045011,-0.085473
1472,0.072914,-0.027008,-0.008552,-0.03959,-0.029038,-0.017206,-0.029507,0.055018,-0.029601,0.076556,...,-0.018427,0.051107,-0.040179,0.043072,-0.14353,0.057518,0.007767,0.100975,0.032745,-0.073464
1664,0.036782,-0.03571,-0.017911,-0.028027,0.004601,0.021264,-0.017391,0.014238,0.024106,0.084602,...,0.04745,0.041513,0.044099,0.03981,-0.056193,0.075596,-0.00476,0.034916,-0.040752,-0.114606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,0.012696,-0.039128,-0.060503,-0.044415,-0.029829,-0.073618,-0.050863,-0.032358,0.009092,0.012477,...,0.008174,0.035563,0.044137,-0.014154,-0.14099,0.049724,-0.020325,0.029315,0.048436,-0.096676
1297,0.067531,0.027873,-0.034303,-0.036806,0.042337,-0.014571,0.047287,0.043312,-0.055092,0.080036,...,-0.019503,0.000634,-0.02769,-0.018206,-0.05588,0.05599,0.013589,0.055061,0.057517,-0.15575
1507,0.113233,-0.01626,-0.039521,0.003618,0.009733,-0.006297,-0.057463,-0.06061,0.013667,0.02138,...,0.038726,0.058563,-0.015468,-0.078145,-0.074099,0.006992,0.033966,0.022923,0.035547,-0.073296
1863,0.015524,0.003294,0.043886,-0.057055,0.00905,0.015464,0.005539,0.021111,-0.016344,0.071674,...,0.042147,0.004722,-0.007199,0.010069,0.006064,0.021039,-0.036758,0.069408,-0.010019,-0.128117


In [166]:
y = y.drop('img_emb', axis=1)
y

,img_emb_000,img_emb_001,img_emb_002,img_emb_003,img_emb_004,img_emb_005,img_emb_006,img_emb_007,img_emb_008,img_emb_009,...,img_emb_502,img_emb_503,img_emb_504,img_emb_505,img_emb_506,img_emb_507,img_emb_508,img_emb_509,img_emb_510,img_emb_511
520,0.134475,2.146872,3.093684,0.25018,0.070831,0.40698,0.484484,0.542378,0.068881,0.325413,...,0.43542,2.844253,0.088756,1.035214,0.782655,0.455288,0.095057,0.694964,0.000367,0.214251
1817,0.41817,2.124481,0.721767,1.234498,1.028805,0.330445,0.594107,0.0,0.194537,0.512329,...,0.267316,0.643107,0.631241,0.061858,0.172819,0.758478,0.899234,1.065097,0.161889,0.503331
188,0.807937,0.23534,0.891588,2.118888,0.964754,0.006434,1.413521,0.03054,0.359847,0.469267,...,0.740372,0.055486,1.068522,0.654895,0.108711,0.787586,1.224116,0.01972,0.002117,1.774309
1472,1.123798,0.129107,0.529932,0.689119,0.088032,0.213432,2.307483,0.219413,0.141571,0.064099,...,0.013426,0.990033,0.037493,0.620104,0.305017,0.716721,0.874736,1.436948,0.54634,0.09154
1664,0.249212,1.338679,1.498986,0.325778,0.181967,0.97209,1.416647,0.378182,0.802813,1.440371,...,0.599462,2.159788,0.637924,1.409396,0.097264,1.439375,0.981709,0.676859,0.021258,0.429731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,0.013477,2.496836,1.286621,0.225792,1.035727,2.823435,0.294912,0.820744,0.01432,1.719165,...,0.477075,0.503651,0.75665,0.045008,0.055604,0.004746,0.259556,0.877341,0.051301,0.180941
1297,2.04283,1.244168,0.09849,0.097462,0.059827,0.002514,3.2317,0.724853,1.240822,0.0,...,0.093601,0.069188,1.52139,0.723119,0.334405,1.313476,0.379416,0.003408,1.218415,0.183199
1507,0.474474,1.141777,0.267572,0.129429,0.914416,0.398639,0.675474,0.519666,0.193633,0.912406,...,1.889288,0.480215,0.631014,1.471817,0.100551,2.064312,1.050679,1.164817,0.556551,0.13919
1863,0.193009,0.261746,0.256704,0.835608,0.385087,0.990576,2.37307,0.206264,0.43979,2.820769,...,0.429085,1.292841,0.032591,1.806116,0.988107,0.091898,1.154531,1.749519,0.163607,1.307694


In [167]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=SEED)
assert len(X) == len(X_train) + len(X_valid)

In [173]:
clf = RandomForestRegressor(random_state=SEED)

In [175]:
clf.fit(X, y)

RandomForestRegressor(random_state=1)

## Сохранение модели

In [176]:
filename = './content/clf_model.sav'
pickle.dump(model, open(filename, 'wb'))

## Выводим результат

In [178]:
dft = pd.read_csv("./content/test.csv")
dft['txt_emb'] = dft['description'].apply(embed_bert_cls0)
Xt = dft[['txt_emb']]
Xt = unpack_column(Xt, 'txt_emb', 312)
Xt = Xt.drop('txt_emb', axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:17<00:00, 52.93it/s]


In [207]:
yt = dft[['id']]
yt.loc[:,'img_emb'] = 0
yt = yt.set_index('id')
yt

C:\Users\Сергей\AppData\Local\Temp\ipykernel_2464\3833680639.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yt.loc[:,'img_emb'] = 0


,img_emb
id,
486,0
813,0
2980,0
13,0
2467,0
...,...
2860,0
2551,0
2399,0


In [208]:
def unpack_column_empty(df, col, num_new_cols):
    
    col_list = [f'{col}_{i:03}' for i in range(num_new_cols)]
    
    val_col = df[col]
    
    df = pd.concat([df,pd.DataFrame(columns=col_list)])
    
    for j in tqdm(range(len(val_col))):
        for i in range(1, num_new_cols+1):
            df.iloc[j, i] = 0
    return df

In [209]:
yt = unpack_column_empty(yt, 'img_emb', 512)

100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:23<00:00, 37.69it/s]


In [210]:
yt

,img_emb,img_emb_000,img_emb_001,img_emb_002,img_emb_003,img_emb_004,img_emb_005,img_emb_006,img_emb_007,img_emb_008,...,img_emb_502,img_emb_503,img_emb_504,img_emb_505,img_emb_506,img_emb_507,img_emb_508,img_emb_509,img_emb_510,img_emb_511
486,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
813,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2980,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2467,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2551,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1492,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [211]:
yt = yt.drop('img_emb', axis=1)

In [212]:
yt

,img_emb_000,img_emb_001,img_emb_002,img_emb_003,img_emb_004,img_emb_005,img_emb_006,img_emb_007,img_emb_008,img_emb_009,...,img_emb_502,img_emb_503,img_emb_504,img_emb_505,img_emb_506,img_emb_507,img_emb_508,img_emb_509,img_emb_510,img_emb_511
486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2467,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2551,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [179]:
pred = clf.predict(Xt)

In [213]:
pred

array([[0.97579215, 0.9954658 , 0.88838344, ..., 1.2250789 , 0.427633  ,
        0.90480314],
       [0.43605052, 2.0195724 , 1.4649509 , ..., 1.34296959, 0.35168188,
        0.82459651],
       [0.60947782, 1.91347794, 1.48290595, ..., 1.35478106, 0.51091309,
        0.69180787],
       ...,
       [0.64579304, 1.43363679, 1.16053112, ..., 1.5507419 , 0.53855392,
        0.98627727],
       [0.76618689, 0.64443499, 1.03879312, ..., 0.61149912, 0.46725661,
        1.66897112],
       [0.63878667, 1.82734182, 1.81285831, ..., 1.38357374, 0.41550425,
        0.74732337]])

In [214]:
yt.iloc[:,:] = pred

In [215]:
yt

,img_emb_000,img_emb_001,img_emb_002,img_emb_003,img_emb_004,img_emb_005,img_emb_006,img_emb_007,img_emb_008,img_emb_009,...,img_emb_502,img_emb_503,img_emb_504,img_emb_505,img_emb_506,img_emb_507,img_emb_508,img_emb_509,img_emb_510,img_emb_511
486,0.975792,0.995466,0.888383,0.561832,0.58228,1.0822,2.044937,0.409188,0.622082,1.325064,...,0.422724,1.35404,0.87062,1.606368,0.646864,0.747736,0.569636,1.225079,0.427633,0.904803
813,0.436051,2.019572,1.464951,0.61288,0.701675,1.009753,1.027126,0.445,0.397849,0.98204,...,0.392588,1.193394,0.556476,0.700075,0.580053,0.709778,0.834011,1.34297,0.351682,0.824597
2980,0.609478,1.913478,1.482906,0.556799,0.767324,1.001255,1.176352,0.585079,0.543992,1.159332,...,0.561276,1.109446,0.571998,0.628807,0.305102,1.02845,0.757888,1.354781,0.510913,0.691808
13,1.013817,1.013027,0.871876,0.579714,0.592979,1.119573,2.057943,0.365126,0.655134,1.36856,...,0.446789,1.403394,0.854028,1.710684,0.651514,0.724811,0.6748,1.382344,0.30339,0.854342
2467,0.717702,1.607706,1.617776,0.556686,0.593557,0.884894,1.420839,0.628076,0.585174,1.244363,...,0.461025,1.467386,0.558371,0.985644,0.337696,0.841253,0.712526,1.544592,0.398361,0.761744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,0.622947,1.876816,1.034411,0.689523,0.760284,0.978856,1.41608,0.482516,0.444136,1.243491,...,0.468395,1.339057,0.74081,0.851441,0.643318,0.732609,0.927908,1.451567,0.550335,0.869578
2551,0.934897,0.392351,0.493885,1.372037,0.71003,0.303537,1.203975,0.5066,1.099427,0.664807,...,0.854144,0.607147,0.859533,1.264792,0.807689,0.990323,0.955724,0.355186,0.346818,0.951483
2399,0.645793,1.433637,1.160531,0.639989,0.689797,1.049212,1.456741,0.503092,0.900865,1.270515,...,0.560117,1.22305,1.085819,0.764855,0.522602,0.807032,1.10045,1.550742,0.538554,0.986277
1492,0.766187,0.644435,1.038793,0.656364,0.455009,0.340663,2.179499,0.466151,0.475174,0.68707,...,0.689159,0.540543,0.641914,1.143424,1.080564,0.461855,0.864421,0.611499,0.467257,1.668971


In [203]:
set_test_img = set(test_img_emb.keys())
#def get_object_img(x):
set_test_img    

{'1002',
 '1005',
 '101',
 '1011',
 '1014',
 '1016',
 '1017',
 '102',
 '1022',
 '1023',
 '1024',
 '1025',
 '1032',
 '1034',
 '1037',
 '1041',
 '1062',
 '1063',
 '1065',
 '1067',
 '107',
 '1081',
 '1085',
 '1087',
 '1089',
 '109',
 '1095',
 '1097',
 '1099',
 '11',
 '1101',
 '1104',
 '1105',
 '1108',
 '1112',
 '1113',
 '1115',
 '1118',
 '112',
 '1120',
 '1123',
 '1124',
 '1131',
 '1132',
 '1134',
 '1135',
 '1137',
 '1138',
 '1141',
 '1143',
 '1149',
 '1150',
 '1151',
 '1152',
 '1154',
 '1156',
 '1158',
 '116',
 '1161',
 '1162',
 '1168',
 '1171',
 '1173',
 '1175',
 '1178',
 '1179',
 '1182',
 '1183',
 '1189',
 '119',
 '1191',
 '1192',
 '1195',
 '1197',
 '1199',
 '1202',
 '1208',
 '1213',
 '1231',
 '1232',
 '1234',
 '1237',
 '1241',
 '1245',
 '1248',
 '1252',
 '1254',
 '1255',
 '1256',
 '126',
 '1260',
 '1265',
 '1266',
 '1269',
 '1273',
 '1275',
 '1279',
 '1280',
 '1281',
 '1285',
 '1288',
 '1292',
 '1293',
 '1296',
 '1299',
 '13',
 '130',
 '1303',
 '1306',
 '1310',
 '1311',
 '1312',
 '131

In [217]:
yt

,img_emb_000,img_emb_001,img_emb_002,img_emb_003,img_emb_004,img_emb_005,img_emb_006,img_emb_007,img_emb_008,img_emb_009,...,img_emb_502,img_emb_503,img_emb_504,img_emb_505,img_emb_506,img_emb_507,img_emb_508,img_emb_509,img_emb_510,img_emb_511
486,0.975792,0.995466,0.888383,0.561832,0.58228,1.0822,2.044937,0.409188,0.622082,1.325064,...,0.422724,1.35404,0.87062,1.606368,0.646864,0.747736,0.569636,1.225079,0.427633,0.904803
813,0.436051,2.019572,1.464951,0.61288,0.701675,1.009753,1.027126,0.445,0.397849,0.98204,...,0.392588,1.193394,0.556476,0.700075,0.580053,0.709778,0.834011,1.34297,0.351682,0.824597
2980,0.609478,1.913478,1.482906,0.556799,0.767324,1.001255,1.176352,0.585079,0.543992,1.159332,...,0.561276,1.109446,0.571998,0.628807,0.305102,1.02845,0.757888,1.354781,0.510913,0.691808
13,1.013817,1.013027,0.871876,0.579714,0.592979,1.119573,2.057943,0.365126,0.655134,1.36856,...,0.446789,1.403394,0.854028,1.710684,0.651514,0.724811,0.6748,1.382344,0.30339,0.854342
2467,0.717702,1.607706,1.617776,0.556686,0.593557,0.884894,1.420839,0.628076,0.585174,1.244363,...,0.461025,1.467386,0.558371,0.985644,0.337696,0.841253,0.712526,1.544592,0.398361,0.761744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,0.622947,1.876816,1.034411,0.689523,0.760284,0.978856,1.41608,0.482516,0.444136,1.243491,...,0.468395,1.339057,0.74081,0.851441,0.643318,0.732609,0.927908,1.451567,0.550335,0.869578
2551,0.934897,0.392351,0.493885,1.372037,0.71003,0.303537,1.203975,0.5066,1.099427,0.664807,...,0.854144,0.607147,0.859533,1.264792,0.807689,0.990323,0.955724,0.355186,0.346818,0.951483
2399,0.645793,1.433637,1.160531,0.639989,0.689797,1.049212,1.456741,0.503092,0.900865,1.270515,...,0.560117,1.22305,1.085819,0.764855,0.522602,0.807032,1.10045,1.550742,0.538554,0.986277
1492,0.766187,0.644435,1.038793,0.656364,0.455009,0.340663,2.179499,0.466151,0.475174,0.68707,...,0.689159,0.540543,0.641914,1.143424,1.080564,0.461855,0.864421,0.611499,0.467257,1.668971


In [219]:
yt['object_img'] = '0' # yt.apply(get_object_img)

In [220]:
yt

,img_emb_000,img_emb_001,img_emb_002,img_emb_003,img_emb_004,img_emb_005,img_emb_006,img_emb_007,img_emb_008,img_emb_009,...,img_emb_503,img_emb_504,img_emb_505,img_emb_506,img_emb_507,img_emb_508,img_emb_509,img_emb_510,img_emb_511,object_img
486,0.975792,0.995466,0.888383,0.561832,0.58228,1.0822,2.044937,0.409188,0.622082,1.325064,...,1.35404,0.87062,1.606368,0.646864,0.747736,0.569636,1.225079,0.427633,0.904803,0
813,0.436051,2.019572,1.464951,0.61288,0.701675,1.009753,1.027126,0.445,0.397849,0.98204,...,1.193394,0.556476,0.700075,0.580053,0.709778,0.834011,1.34297,0.351682,0.824597,0
2980,0.609478,1.913478,1.482906,0.556799,0.767324,1.001255,1.176352,0.585079,0.543992,1.159332,...,1.109446,0.571998,0.628807,0.305102,1.02845,0.757888,1.354781,0.510913,0.691808,0
13,1.013817,1.013027,0.871876,0.579714,0.592979,1.119573,2.057943,0.365126,0.655134,1.36856,...,1.403394,0.854028,1.710684,0.651514,0.724811,0.6748,1.382344,0.30339,0.854342,0
2467,0.717702,1.607706,1.617776,0.556686,0.593557,0.884894,1.420839,0.628076,0.585174,1.244363,...,1.467386,0.558371,0.985644,0.337696,0.841253,0.712526,1.544592,0.398361,0.761744,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,0.622947,1.876816,1.034411,0.689523,0.760284,0.978856,1.41608,0.482516,0.444136,1.243491,...,1.339057,0.74081,0.851441,0.643318,0.732609,0.927908,1.451567,0.550335,0.869578,0
2551,0.934897,0.392351,0.493885,1.372037,0.71003,0.303537,1.203975,0.5066,1.099427,0.664807,...,0.607147,0.859533,1.264792,0.807689,0.990323,0.955724,0.355186,0.346818,0.951483,0
2399,0.645793,1.433637,1.160531,0.639989,0.689797,1.049212,1.456741,0.503092,0.900865,1.270515,...,1.22305,1.085819,0.764855,0.522602,0.807032,1.10045,1.550742,0.538554,0.986277,0
1492,0.766187,0.644435,1.038793,0.656364,0.455009,0.340663,2.179499,0.466151,0.475174,0.68707,...,0.540543,0.641914,1.143424,1.080564,0.461855,0.864421,0.611499,0.467257,1.668971,0


## Осталось по эмбеддингу определить ближайшую к нему картинку

In [227]:
vec = yt.iloc[0, :-1].tolist()
print(vec)

[0.9757921541109681, 0.9954658003151416, 0.8883834371436388, 0.5618320281244814, 0.5822801474697189, 1.0821999527607113, 2.0449371613562106, 0.4091875808491659, 0.6220822233892978, 1.3250640472769737, 0.41403448218014094, 0.8854845395125449, 0.73725187972188, 0.3668358976603486, 0.6431144664785825, 1.288777324706316, 0.36293400967493655, 0.919951611738652, 0.9524254994094372, 1.370743470862508, 0.24895526240579785, 0.7329404549160973, 1.0285081880749203, 1.0252056827768683, 0.7184733351669275, 0.5627537739928812, 0.7419668355211616, 1.5335264691710473, 0.4920335984649137, 0.6868094163388014, 1.4292147482931614, 0.5958492176048458, 0.32870284633245317, 1.595828163968399, 0.7473736255289987, 0.48559003327813116, 1.3726005623489619, 0.8975739200692624, 1.8067977421171963, 0.6141501146019437, 0.609786709533073, 0.7432013929978711, 0.5201949866116047, 1.5442014756659046, 0.827100283280015, 1.1568194691836835, 0.9083395303040743, 0.6515575237781741, 0.6581936910876539, 1.2268149662483483, 0.

In [236]:
#from numpy import dot
#from numpy.linalg import norm

set_test_img = set(test_img_emb.keys())
for i in tqdm(range(len(yt))):
    pvec = yt.iloc[i, :-1].tolist()
    min_diff = -1.1
    find_v = ''
    for v in set_test_img:
        #cs = dot(pvec, test_img_emb[v])/(norm(pvec)*norm(test_img_emb[v]))
        cs = 1 - spatial.distance.cosine(pvec, test_img_emb[v])
        if cs > min_diff:
            min_diff = cs
            find_v = v
    #print(min_diff)
    yt.iloc[i, -1] = find_v 
    set_test_img.remove(find_v)
    #print(vec)
yt

  0%|▏                                                                                 | 2/900 [00:00<01:11, 12.57it/s]

0.8940225290144488
0.896326418326703
0.8839143969332209


  1%|▌                                                                                 | 6/900 [00:00<01:20, 11.12it/s]

0.8943821343333663
0.8806322997599585
0.8828666572175455


  1%|▋                                                                                 | 8/900 [00:00<01:15, 11.81it/s]

...

 93%|█████████████████████████████████████████████████████████████████████████▍     | 837/900 [00:32<00:00, 121.59it/s]

0.7459070816110593
0.820508969313396
0.7791249768970403
0.8249973065882438
0.7765114503168221
0.718221185556544
0.8655706662756969
0.7379201957656067
0.7514913085019133
0.7200601950834304
0.822056397481296
0.7264691056429654
0.7135882422966474
0.7799591863824884
0.7673012566172176
0.807876612043204
0.8491751022518356
0.7291930510377487
0.7587539638080497
0.823259041117459
0.7418629460533037
0.7445957611870185
0.7500333013289365
0.8599821621557855
0.7435459442061625
0.7921204866222629
0.7709044184097628
0.7890983588069153
0.8300969676694121
0.9047486552136005
0.790563803971127


100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:32<00:00, 27.56it/s]

0.7764127269592561
0.7521581396285232
0.7167667855647195
0.7242146646208604
0.8098220809868241
0.7714970944688155
0.7271657861898222
0.7195632816515093
0.7826785328014058
0.8120685768998672
0.9163489317645859
0.696466250323694
0.7551790295348313
0.7214512233889954
0.6741628387168882
0.7474412008843797
0.7175373021745288
0.8092858041461766
0.7640558950919561
0.7541976964028484
0.8326164915513264
0.6912544688155483
0.7171395709749709
0.6986769617477121
0.7135269145066219
0.7649037527290509
0.7455393522437863
0.8177753183208909
0.7724453893626805
0.7734894657304107
0.6867337670317541
0.7336405725580701
0.6889048996352952
0.6824583447327744
0.7220433213633587
0.6545562722893525
0.8009231683931068
0.7379892326067612
0.7434579323290487
0.7646797487465503
0.639774510447742
0.7755060698251447
0.6989639463591591
0.7431842812041071
0.6537052518330044
0.7141871043542719
0.7047182748644595
0.6745407335729758
0.7051634997388898
0.7376025254810743
0.6276588456706295
0.682359955513176
0.7550234913033

,img_emb_000,img_emb_001,img_emb_002,img_emb_003,img_emb_004,img_emb_005,img_emb_006,img_emb_007,img_emb_008,img_emb_009,...,img_emb_503,img_emb_504,img_emb_505,img_emb_506,img_emb_507,img_emb_508,img_emb_509,img_emb_510,img_emb_511,object_img
486,0.975792,0.995466,0.888383,0.561832,0.58228,1.0822,2.044937,0.409188,0.622082,1.325064,...,1.35404,0.87062,1.606368,0.646864,0.747736,0.569636,1.225079,0.427633,0.904803,1241
813,0.436051,2.019572,1.464951,0.61288,0.701675,1.009753,1.027126,0.445,0.397849,0.98204,...,1.193394,0.556476,0.700075,0.580053,0.709778,0.834011,1.34297,0.351682,0.824597,1652
2980,0.609478,1.913478,1.482906,0.556799,0.767324,1.001255,1.176352,0.585079,0.543992,1.159332,...,1.109446,0.571998,0.628807,0.305102,1.02845,0.757888,1.354781,0.510913,0.691808,1491
13,1.013817,1.013027,0.871876,0.579714,0.592979,1.119573,2.057943,0.365126,0.655134,1.36856,...,1.403394,0.854028,1.710684,0.651514,0.724811,0.6748,1.382344,0.30339,0.854342,1208
2467,0.717702,1.607706,1.617776,0.556686,0.593557,0.884894,1.420839,0.628076,0.585174,1.244363,...,1.467386,0.558371,0.985644,0.337696,0.841253,0.712526,1.544592,0.398361,0.761744,1492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2860,0.622947,1.876816,1.034411,0.689523,0.760284,0.978856,1.41608,0.482516,0.444136,1.243491,...,1.339057,0.74081,0.851441,0.643318,0.732609,0.927908,1.451567,0.550335,0.869578,2986
2551,0.934897,0.392351,0.493885,1.372037,0.71003,0.303537,1.203975,0.5066,1.099427,0.664807,...,0.607147,0.859533,1.264792,0.807689,0.990323,0.955724,0.355186,0.346818,0.951483,2365
2399,0.645793,1.433637,1.160531,0.639989,0.689797,1.049212,1.456741,0.503092,0.900865,1.270515,...,1.22305,1.085819,0.764855,0.522602,0.807032,1.10045,1.550742,0.538554,0.986277,1065
1492,0.766187,0.644435,1.038793,0.656364,0.455009,0.340663,2.179499,0.466151,0.475174,0.68707,...,0.540543,0.641914,1.143424,1.080564,0.461855,0.864421,0.611499,0.467257,1.668971,644


In [237]:
yt = yt[['object_img']]

In [239]:
yt = yt.reset_index()

,index,object_img
0,486,1241
1,813,1652
2,2980,1491
3,13,1208
4,2467,1492
...,...,...
895,2860,2986
896,2551,2365
897,2399,1065
898,1492,644


In [240]:
yt.columns = ['id', 'object_img']

In [241]:
yt

,id,object_img
0,486,1241
1,813,1652
2,2980,1491
3,13,1208
4,2467,1492
...,...,...
895,2860,2986
896,2551,2365
897,2399,1065
898,1492,644


In [242]:
yt.to_csv("./content/RSF_submission_final.csv", index=False)